# **FastText**

Sumber bacaan: https://fasttext.cc/

In [ ]:
!pip install fasttext

## Text Classification

Bahan main hari ini: https://fasttext.cc/docs/en/supervised-tutorial.html

1. Ambil dulu data yang akan diolah. 

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz

In [ ]:
!head cooking.stackexchange.txt

Sebelum melakukan *training* untuk klasifier pertama, kita harus lakukan **split data** ke **train** dan **validation**. 

In [ ]:
#wc = word count
!wc cooking.stackexchange.txt

Dataset kita berisi 15404 data contoh. Nah kita bagi 12404 sebagai **training set** dan 3000 sebagai **validation set**.

In [ ]:
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

### Our first classifier
Saatnya melakukan **train** untuk **classifier** pertama kita

In [ ]:
import fasttext
model = fasttext.train_supervised(input="cooking.train")

In [ ]:
#simpan model kita supaya nanti tinggal panggil saja
model.save_model("model_cooking.bin")

In [ ]:
#Saatnya coba klasifier kita

model.predict("Which baking dish is best to bake a banana bread ?")

In [ ]:
model.predict("Why not put knives in the dishwasher?")

In [ ]:
model.predict("Kenapa gak ada garam di dapur?")

Label prediksinya adalah **food-safety** untuk kalimat dalam bahasa Indonesia, artinya ini tidak/kurang relevan. Terlihat kalau model ini gagal untuk contoh sederhana.

Supaya lebih baik hasilnya, kita coba tes **validation data**-nya dengan menulis koding berikut:

In [ ]:
model.test("cooking.valid")

Keluarannya jumlah samples (ada 3000), nilai presisi (**precision**) adalah satu (0.135) dan nilai **recall**-nya adalah satu (0.0583).
Kita juga bisa lakukan komputasi dengan **precision** 5 dan **recall** 5:

In [ ]:
model.test("cooking.valid", k=5)

**Advanced readers: precision and recall**
Presisi (*precision*) adalah angka dari benar label dibandingkan dengan label prediksi oleh **fastText**. Kalau *recall* adalah angka label yang sukses diprediksi, diantara semua label yang real. Perhatikan contoh kalimat berikut:

*Why not put knives in the dishwasher?*

Dalam Stack Exchange, kalimat ini di-label-i 3 tags: *equipment*, *cleaning* dan *knives*. Prediksi 5 label tertinggi oleh model berisi: *food-safety*, *baking*, *bread*, *substitutions* dan *equipment*.

In [ ]:
model.predict("Why not put knives in the dishwasher?", k=5)

### Making the model better

Model yang dibuat oleh **fastText** untuk klasifikasi pertama sangat buruk. Mari kita coba improvisasi *performance*-nya dengan mengganti parameter default-nya. 

#### preprocessing the data

Pada data yang ada, terdapat huruf besar dan tanda baca. Nah langkah pertama kita adalah buat supaya semua menjadi huruf kecil dan menghapus tanda baca supaya meningkatkan performansi dari model kita. Gunakan ***sed*** dan ***tr***:

In [ ]:
!cat cooking.stackexchange.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > cooking.preprocessed.txt
!head -n 12404 cooking.preprocessed.txt > cooking.train
!tail -n 3000 cooking.preprocessed.txt > cooking.valid

Saatnya melatih model baru kita dengan pre-processed data:

In [ ]:
import fasttext
model = fasttext.train_supervised(input="cooking.train")

#### more epochs and larger learning rate

Secara default, fastText melihat setiap contoh *training* hanya 5X selama proses *training*, terlalu kecil, artinya **training set** kita hanya punya 12 ribu contoh *training*. Jumlah waktunya untuk tiap contoh (dikenal dengan **number of epochs**), dapat kita naikan/***increased*** menggunakan opsi **-epoch**:

In [ ]:
import fasttext
model = fasttext.train_supervised(input="cooking.train", epoch=25)

Mari kita coba model baru:

In [ ]:
model.test("cooking.valid")

Yeay! hasilnya jauh lebih baik! Ini merupakan cara lain untuk meningkatkan **learning speed** dari model kita, bisa bertambah atau berkurang juga sih (increase or decrease). Nilai **learning rate** = 0, berarti **model** tidak berubah sama sekali, artinya model tidak belajar apapun. Nilai **learning rate** yang bagus adalah dalam *range* **0.1 - 1.0**.

In [ ]:
model = fasttext.train_supervised(input="cooking.train", lr=1.0)

In [ ]:
model.test("cooking.valid")

Yeay! Nilainya juga jauh lebih baik. Sekarang kita coba kedua hal tersebut bersamaan:

In [ ]:
model = fasttext.train_supervised(input="cooking.train", lr=1.0, epoch=25)

In [ ]:
model.test("cooking.valid")

#### word n-grams

Oke, sekarang kita lanjut tingkatkan performansi **model** kita dengan menggunakan **word bigrams**. Ini cocok banget untuk klasifikasi seperti ***sentiment analysis***.

In [ ]:
model = fasttext.train_supervised(input="cooking.train", lr=1.0, epoch=25, wordNgrams=2)

In [ ]:
model.test("cooking.valid")

Dengan beberapa langkah, kita bisa naikan nilai presisi menjadi 59.9%. Langkah kita tadi adalah:
*   **preprocessing the data**;
*   ubah nilai **epochs** (gunakan **-epoch**, standard range [5 - 50]);
*   ubah nilai **learning rate** (gunakan **-lr**, standard range [0.1 - 1.0]);
*   gunakan word n-grams (**-wordNgrams**, standard range [1 - 5].





### Scaling things up

Saat kita training model kita yang berisi ribuan data, hanya membutuhkan beberapa detik saja. Tapi, apabila dataset kita sangat besar maka butuh waktu yang lumayan lama untuk training dataset kita dengan pe-label-an. Solusinya adalah dengan menggunakan **hierarchical softmax**. Hal ini dapat dilakukan dengan menggunakan opsi **-loss hs**:

In [ ]:
model = fasttext.train_supervised(input="cooking.train", lr=1.0, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')

The hierarchical softmax is a loss function that approximates the softmax with a much faster computation.

The idea is to build a binary tree whose leaves correspond to the labels. Each intermediate node has a binary decision activation (e.g. sigmoid) that is trained, and predicts if we should go to the left or to the right. The probability of the output unit is then given by the product of the probabilities of intermediate nodes along the path from the root to the output unit leave.

For a detailed explanation, you can have a look on [this video](https://www.youtube.com/watch?v=B95LTf2rVWM).

In fastText, we use a Huffman tree, so that the lookup time is faster for more frequent outputs and thus the average lookup time for the output is optimal.

### **Multi-label classification**

Main-main dengan **multiple labels**, kita tetap dapat menggunakan **softmax loss** juga dengan mengubah parameter untuk **prediction**. 
Cara paling aman untuk menangani ***multiple labels*** adalah dengan menggunakan **independent binary classifiers** untuk masing-masing label. Gunakan **-loss one-vs-all** atau **-loss ova**.

In [ ]:
import fasttext
model = fasttext.train_supervised(input="cooking.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')

Ide tepat untuk mengurangi **learning rate** dibandingkan kehilangan **loss functions**.

Sekarang kita lihat prediksinya, kita ingin prediksi sebaik mungkin (argument -1) dan kita ingin hanya label dengan nilai probabilitas tertinggi atau sama dengan 0.5:

In [ ]:
model.predict("Which baking dish is best to bake a banana bread ?", k=-1, threshold=0.5)

In [ ]:
model.test("cooking.valid", k=-1)

## Word representations

Sumber bacaan: https://fasttext.cc/docs/en/unsupervised-tutorial.html

### Getting the data

Untuk main dengan **word vectors**, kita butuh **text corpus ** yang sangat besar. Nah sekarang kita main dengan artikel di Wikipedia tapi kita juga bisa gunakan sumber yang lainnya ([contoh lain](https://statmt.org/)). Untuk mengunduh file Wikipedia, jalankan koding berikut:

In [ ]:
!wget https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2